# 标的获取

In [58]:
import numpy as mp
import pandas as pd
import os
import jqdatasdk
import datetime
import json

In [59]:
jqdatasdk.auth('13350103318', '87654321wW')
# jqdatasdk.auth("18867905508", "Ljy19950217")

In [80]:
jqdatasdk.get_query_count()

{'total': 500000, 'spare': 29758}

In [61]:
with open('hs300_cleared_dict.json', 'r') as f:
    symbol_dict = json.load(f)

In [63]:
from dateutil.relativedelta import relativedelta
dateList = list(symbol_dict.keys())
start = datetime.datetime.strptime(dateList[0], '%Y-%m-%d') - relativedelta(months=1)  # 多拉一个月的数据进行因子构造
start = start.strftime('%Y-%m-%d')
end = dateList[-1]  # 指定后复权参照日
last_symbol = []

In [64]:
subdateList = dateList[27:35]
print(subdateList[0], '-', subdateList[-1])

2019-03-31 - 2019-10-31


In [77]:
fields = ['date', 'open', 'close', 'high', 'low', 'volume']
data = pd.DataFrame()
for date in subdateList:
    print(date)
    symbol_list = symbol_dict[date]
    symbol_list = list(set(symbol_list).union(last_symbol))  # 上期在资产池中的标的也需要获得这一期的交易数据
    last_symbol = symbol_dict[date]
    next_month = datetime.datetime.strptime(date, '%Y-%m-%d') + relativedelta(months=1)  # 获取下个月前近似两个月的交易数据
    temp = jqdatasdk.get_bars(security=symbol_list, fields=fields, end_dt=next_month, count=704, fq_ref_date=end, unit='15m', df=True)
    data = data.append(temp)
data.head()

2019-03-31
2019-04-30
2019-05-31
2019-06-30
2019-07-31
2019-08-31
2019-09-30
2019-10-31


date  open  close  high   low       volume
000630.XSHE 0 2019-02-26 10:30:00  2.29   2.32  2.42  2.24  200026887.0
            1 2019-02-26 11:30:00  2.31   2.35  2.35  2.27   52355033.0
            2 2019-02-26 14:00:00  2.35   2.36  2.38  2.35   46943937.0
            3 2019-02-26 15:00:00  2.36   2.30  2.36  2.28   63932403.0
            4 2019-02-27 10:30:00  2.26   2.29  2.34  2.25   69608157.0

In [79]:
'''写入本地，方便之后调用'''
# jsonObj = json.dumps(data)
# fileObj = open('data/minute_data_%s_%s.json' % (subdateList[0].replace('-', ''), subdateList[-1].replace('-', '')), 'w', encoding='utf-8')
# fileObj.write(jsonObj)
# fileObj.close()
data.to_csv('data/minute_data_%s_%s.csv' % (subdateList[0].replace('-', ''), subdateList[-1].replace('-', '')))

In [52]:
'minute_data_%s_%s.json' % (subdateList[0].replace('-', ''), subdateList[-1].replace('-', ''))

'minute_data_20190331_20191031.json'